# 2 Layer Neural Network

## Imports

In [1]:
import os
import pandas as pd

## Import data and create .csv

In [2]:
# Directory to MRI Images
base_dir = '/Users/benrandoing/Downloads/archive/AugmentedAlzheimerDataset'
classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

if not os.path.exists('dataset.csv'):
    data = []

    for label, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            data.append([image_path, label])

    df = pd.DataFrame(data, columns=['path', 'label'])
    df.to_csv('dataset.csv', index=False)

## Data Loading and Pre-Processing

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.dataframe = pd.read_csv(csv_file)
        self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)  # Shuffle the dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = int(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224 X 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load the entire dataset
dataset = CustomDataset(csv_file='dataset.csv', transform=transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [4]:
print(f'Training Samples: {len(train_dataset)}')
print(f'Validation Samples: {len(val_dataset)}')
print(f'Test Samples: {len(test_dataset)}')

Training Samples: 23788
Validation Samples: 5097
Test Samples: 5099


In [5]:
print(train_dataset)

In [6]:
import matplotlib.pyplot as plt
#plt.plot(train_dataset[0][0])
#plt.show()

## Pytorch Logistic Regression Model

In [7]:
class FiveLayerNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim):
        super(FiveLayerNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim1)
        self.fc2 = torch.nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = torch.nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = torch.nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = torch.nn.Linear(hidden_dim4, output_dim)

    def forward(self, x):
        #x = x.view(-1, 224 * 224 * 3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

input_dim = 224 * 224 * 3  # Assuming the images are RGB
hidden_dim1 = 512
hidden_dim2 = 256
hidden_dim3 = 128
hidden_dim4 = 64
output_dim = 4  # Four classes

#model = FiveLayerNN(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim)
import torchvision
from torchvision.models import resnet18, ResNet18_Weights
model = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

model.fc = torch.nn.Linear(model.fc.in_features, output_dim)


## Training Loop

In [8]:
learning_rate = 0.001
num_epochs = 20
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.RMSProp(model.parameters(), lr=learning_rate)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    i = 0
    for inputs, labels in train_loader:
        i = i + 1
        print("Training continues..", total_loss, "current i = ", i)    
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    #lr decay
    #lr_scheduler.step()

    print(f"Iteration {epoch} - Loss = {total_loss / len(train_loader)}")

Training continues.. 0.0 current i =  1
Training continues.. 1.6307330131530762 current i =  2
Training continues.. 3.2073309421539307 current i =  3
Training continues.. 4.853025197982788 current i =  4
Training continues.. 6.367113947868347 current i =  5
Training continues.. 7.901331305503845 current i =  6
Training continues.. 9.386110186576843 current i =  7
Training continues.. 10.93295967578888 current i =  8
Training continues.. 12.34346067905426 current i =  9
Training continues.. 13.75242030620575 current i =  10
Training continues.. 15.21662175655365 current i =  11
Training continues.. 16.652053833007812 current i =  12
Training continues.. 18.121073246002197 current i =  13
Training continues.. 19.543941259384155 current i =  14
Training continues.. 20.95519518852234 current i =  15
Training continues.. 22.362087726593018 current i =  16
Training continues.. 23.700162887573242 current i =  17
Training continues.. 25.144147872924805 current i =  18
Training continues.. 26.6

Training continues.. 192.68878591060638 current i =  149
Training continues.. 193.84395372867584 current i =  150
Training continues.. 195.00427544116974 current i =  151
Training continues.. 196.15816175937653 current i =  152
Training continues.. 197.3007777929306 current i =  153
Training continues.. 198.50098764896393 current i =  154
Training continues.. 199.69366884231567 current i =  155
Training continues.. 200.8986370563507 current i =  156
Training continues.. 202.10275423526764 current i =  157
Training continues.. 203.28577268123627 current i =  158
Training continues.. 204.3817709684372 current i =  159
Training continues.. 205.48794376850128 current i =  160
Training continues.. 206.5996071100235 current i =  161
Training continues.. 207.73042273521423 current i =  162
Training continues.. 208.86512792110443 current i =  163
Training continues.. 210.04016971588135 current i =  164
Training continues.. 211.18110620975494 current i =  165
Training continues.. 212.2643798589

Training continues.. 351.4909372329712 current i =  295
Training continues.. 352.5393656492233 current i =  296
Training continues.. 353.4918128848076 current i =  297
Training continues.. 354.43661457300186 current i =  298
Training continues.. 355.45606476068497 current i =  299
Training continues.. 356.44279307127 current i =  300
Training continues.. 357.41562682390213 current i =  301
Training continues.. 358.3920730948448 current i =  302
Training continues.. 359.396992623806 current i =  303
Training continues.. 360.433728992939 current i =  304
Training continues.. 361.4209741950035 current i =  305
Training continues.. 362.43773716688156 current i =  306
Training continues.. 363.39468532800674 current i =  307
Training continues.. 364.4221667647362 current i =  308
Training continues.. 365.3492262363434 current i =  309
Training continues.. 366.37109541893005 current i =  310
Training continues.. 367.3713297843933 current i =  311
Training continues.. 368.3808763027191 current

Training continues.. 61.517051696777344 current i =  70
Training continues.. 62.32063591480255 current i =  71
Training continues.. 63.18130201101303 current i =  72
Training continues.. 64.04330402612686 current i =  73
Training continues.. 64.98401302099228 current i =  74
Training continues.. 65.85789406299591 current i =  75
Training continues.. 66.58901643753052 current i =  76
Training continues.. 67.41301888227463 current i =  77
Training continues.. 68.35353064537048 current i =  78
Training continues.. 69.21391522884369 current i =  79
Training continues.. 70.05867266654968 current i =  80
Training continues.. 70.90733247995377 current i =  81
Training continues.. 71.76063513755798 current i =  82
Training continues.. 72.74090188741684 current i =  83
Training continues.. 73.60728973150253 current i =  84
Training continues.. 74.4318996667862 current i =  85
Training continues.. 75.20890456438065 current i =  86
Training continues.. 76.02712523937225 current i =  87
Training c

Training continues.. 182.308464884758 current i =  216
Training continues.. 183.11007928848267 current i =  217
Training continues.. 183.93234181404114 current i =  218
Training continues.. 184.69135576486588 current i =  219
Training continues.. 185.52713978290558 current i =  220
Training continues.. 186.40293592214584 current i =  221
Training continues.. 187.2554448246956 current i =  222
Training continues.. 188.13642972707748 current i =  223
Training continues.. 188.8992092013359 current i =  224
Training continues.. 189.69418269395828 current i =  225
Training continues.. 190.57713878154755 current i =  226
Training continues.. 191.44061130285263 current i =  227
Training continues.. 192.21091043949127 current i =  228
Training continues.. 192.9894858598709 current i =  229
Training continues.. 193.75824826955795 current i =  230
Training continues.. 194.70731234550476 current i =  231
Training continues.. 195.60293400287628 current i =  232
Training continues.. 196.27405506372

...

Training continues.. 1.9781238492578268 current i =  37
Training continues.. 2.068679539486766 current i =  38
Training continues.. 2.1263911817222834 current i =  39
Training continues.. 2.212070832028985 current i =  40
Training continues.. 2.2490799073129892 current i =  41
Training continues.. 2.328587869182229 current i =  42
Training continues.. 2.365891171619296 current i =  43
Training continues.. 2.4576169159263372 current i =  44
Training continues.. 2.514177141711116 current i =  45
Training continues.. 2.57889018766582 current i =  46
Training continues.. 2.616083288565278 current i =  47
Training continues.. 2.65509289316833 current i =  48
Training continues.. 2.702758966013789 current i =  49
Training continues.. 2.7465128768235445 current i =  50
Training continues.. 2.8669830728322268 current i =  51
Training continues.. 2.9135790709406137 current i =  52
Training continues.. 2.9887571278959513 current i =  53
Training continues.. 3.045075537636876 current i =  54
Trai

Training continues.. 9.82601448521018 current i =  185
Training continues.. 9.87500337138772 current i =  186
Training continues.. 9.967544179409742 current i =  187
Training continues.. 9.999190475791693 current i =  188
Training continues.. 10.04617989435792 current i =  189
Training continues.. 10.094818782061338 current i =  190
Training continues.. 10.129804488271475 current i =  191
Training continues.. 10.191509373486042 current i =  192
Training continues.. 10.250910051167011 current i =  193
Training continues.. 10.287821892648935 current i =  194
Training continues.. 10.315818870440125 current i =  195
Training continues.. 10.406957836821675 current i =  196
Training continues.. 10.459433829411864 current i =  197
Training continues.. 10.552130265161395 current i =  198
Training continues.. 10.597430588677526 current i =  199
Training continues.. 10.652808716520667 current i =  200
Training continues.. 10.701773842796683 current i =  201
Training continues.. 10.74484722130000

Training continues.. 17.211803460493684 current i =  330
Training continues.. 17.2385686468333 current i =  331
Training continues.. 17.3010162319988 current i =  332
Training continues.. 17.369895620271564 current i =  333
Training continues.. 17.417421305552125 current i =  334
Training continues.. 17.46312797628343 current i =  335
Training continues.. 17.52440215088427 current i =  336
Training continues.. 17.585744535550475 current i =  337
Training continues.. 17.62016154266894 current i =  338
Training continues.. 17.679215783253312 current i =  339
Training continues.. 17.73061959259212 current i =  340
Training continues.. 17.76782546751201 current i =  341
Training continues.. 17.8244862947613 current i =  342
Training continues.. 17.861806308850646 current i =  343
Training continues.. 17.893420899286866 current i =  344
Training continues.. 17.926783418282866 current i =  345
Training continues.. 17.961863121017814 current i =  346
Training continues.. 18.00849541462958 cur

Training continues.. 4.301430102437735 current i =  105
Training continues.. 4.32748388312757 current i =  106
Training continues.. 4.369384003803134 current i =  107
Training continues.. 4.408166417852044 current i =  108
Training continues.. 4.536260822787881 current i =  109
Training continues.. 4.567994924262166 current i =  110
Training continues.. 4.613315815106034 current i =  111
Training continues.. 4.660245856270194 current i =  112
Training continues.. 4.692316012457013 current i =  113
Training continues.. 4.737981604412198 current i =  114
Training continues.. 4.773126041516662 current i =  115
Training continues.. 4.85720438323915 current i =  116
Training continues.. 4.8824416641145945 current i =  117
Training continues.. 4.917558642104268 current i =  118
Training continues.. 4.969922250136733 current i =  119
Training continues.. 4.993820484727621 current i =  120
Training continues.. 5.036573030054569 current i =  121
Training continues.. 5.061234226450324 current i 

Training continues.. 10.347385942004621 current i =  252
Training continues.. 10.384886565618217 current i =  253
Training continues.. 10.514739307574928 current i =  254
Training continues.. 10.556642211042345 current i =  255
Training continues.. 10.590315795503557 current i =  256
Training continues.. 10.666221438907087 current i =  257
Training continues.. 10.714801552705467 current i =  258
Training continues.. 10.733254401944578 current i =  259
Training continues.. 10.808892480097711 current i =  260
Training continues.. 10.845273378305137 current i =  261
Training continues.. 10.881108011119068 current i =  262
Training continues.. 10.92212954070419 current i =  263
Training continues.. 10.950771453790367 current i =  264
Training continues.. 10.993622574023902 current i =  265
Training continues.. 11.034436496905982 current i =  266
Training continues.. 11.072669825516641 current i =  267
Training continues.. 11.104300535283983 current i =  268
Training continues.. 11.13886155

## Evaluation

In [9]:
# Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

validation_accuracy = 100 * correct / total
print(f"Validation Accuracy: {validation_accuracy}%")

Validation Accuracy: 93.54522268000785%


In [10]:
# Testing loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy}%")

Test Accuracy: 93.97921161011963%
